# 3. Modelagem e Avaliação Comparativa

Este notebook realiza o **Treinamento de Três Modelos** e a **Avaliação com Três Métricas** para o problema de regressão de preço de imóveis, conforme a Parte 3 da avaliação.

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os

# Caminho para os dados pré-processados
DATA_PATH = "../data/processed_data.npz"
MODEL_PATH = "../modelo_final.pkl"

# 1. Carregamento dos Dados
print("--- 1. Carregamento dos Dados ---")
try:
    data = np.load(DATA_PATH, allow_pickle=True)
    X_train = pd.DataFrame(data['X_train'], columns=data['feature_names'])
    X_test = pd.DataFrame(data['X_test'], columns=data['feature_names'])
    y_train = data['y_train']
    y_test = data['y_test']
    print("Dados de treino e teste carregados com sucesso.")
except FileNotFoundError:
    print(f"Erro: Arquivo não encontrado em {DATA_PATH}.")
    exit()

# 2. Definição dos Modelos
print("\n--- 2. Definição dos Modelos ---")
models = {
    "Regressão Linear": LinearRegression(),
    "Ridge": Ridge(alpha=1.0), # alpha padrão
    "Árvore de Decisão": DecisionTreeRegressor(random_state=42)
}
print(f"Modelos a serem treinados: {list(models.keys())}")

# 3. Treinamento e Avaliação
print("\n--- 3. Treinamento e Avaliação ---")
results = []
best_r2 = -np.inf
best_model = None
best_model_name = ""

for name, model in models.items():
    print(f"Treinando {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Cálculo das Métricas
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    results.append({
        'Modelo': name,
        'R² (R-squared)': r2,
        'MAE (Mean Absolute Error)': mae,
        'RMSE (Root Mean Squared Error)': rmse
    })
    
    if r2 > best_r2:
        best_r2 = r2
        best_model = model
        best_model_name = name

results_df = pd.DataFrame(results)

# 4. Análise Comparativa dos Resultados
print("\n--- 4. Análise Comparativa dos Resultados ---")
print(results_df.sort_values(by='R² (R-squared)', ascending=False))
print(f"\nO melhor modelo (baseado no R²) é: {best_model_name}")

# 5. Salvando o Melhor Modelo (Parte 4.1)
print("\n--- 5. Salvando o Melhor Modelo ---")
joblib.dump(best_model, MODEL_PATH)
print(f"Modelo '{best_model_name}' salvo em {MODEL_PATH}")

# 6. Demonstração de Uso (Parte 4.2)
print("\n--- 6. Demonstração de Uso (Previsão) ---")
loaded_model = joblib.load(MODEL_PATH)
print(f"Modelo carregado: {loaded_model.__class__.__name__}")

# Pegando a primeira amostra do conjunto de teste para demonstração
sample_X = X_test.iloc[0].values.reshape(1, -1)
sample_y_true = y_test[0]
sample_y_pred = loaded_model.predict(sample_X)[0]

print(f"Previsão para a primeira amostra de teste:")
print(f"  Valor Real: R$ {sample_y_true:,.2f}")
print(f"  Previsão do Modelo: R$ {sample_y_pred:,.2f}")

# Salvando o código do notebook em um arquivo .py para execução no shell
with open('../notebooks/03_Modelagem_e_Avaliacao.py', 'w') as f:
    f.write("""import pandas as pd\nimport numpy as np\nimport joblib\nfrom sklearn.linear_model import LinearRegression, Ridge\nfrom sklearn.tree import DecisionTreeRegressor\nfrom sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score\nimport os\n\nDATA_PATH = "data/processed_data.npz"\nMODEL_PATH = "modelo_final.pkl"\n\n# 1. Carregamento dos Dados\ntry:\n    data = np.load(DATA_PATH, allow_pickle=True)\n    X_train = pd.DataFrame(data['X_train'], columns=data['feature_names'])\n    X_test = pd.DataFrame(data['X_test'], columns=data['feature_names'])\n    y_train = data['y_train']\n    y_test = data['y_test']\nexcept FileNotFoundError:\n    print(f"Erro: Arquivo não encontrado em {DATA_PATH}.")\n    exit()\n\n# 2. Definição dos Modelos\nmodels = {\n    "Regressão Linear": LinearRegression(),\n    "Ridge": Ridge(alpha=1.0),\n    "Árvore de Decisão": DecisionTreeRegressor(random_state=42)\n}\n\n# 3. Treinamento e Avaliação\nresults = []\nbest_r2 = -np.inf\nbest_model = None\nbest_model_name = ""\n\nfor name, model in models.items():\n    model.fit(X_train, y_train)\n    y_pred = model.predict(X_test)\n    \n    r2 = r2_score(y_test, y_pred)\n    mae = mean_absolute_error(y_test, y_pred)\n    rmse = np.sqrt(mean_squared_error(y_test, y_pred))\n    \n    results.append({\n        'Modelo': name,\n        'R² (R-squared)': r2,\n        'MAE (Mean Absolute Error)': mae,\n        'RMSE (Root Mean Squared Error)': rmse\n    })\n    \n    if r2 > best_r2:\n        best_r2 = r2\n        best_model = model\n        best_model_name = name\n\nresults_df = pd.DataFrame(results)\n\n# 4. Análise Comparativa dos Resultados\nprint("Análise Comparativa:")\nprint(results_df.sort_values(by='R² (R-squared)', ascending=False).to_markdown(index=False))\nprint(f"\nO melhor modelo (baseado no R²) é: {best_model_name}")\n\n# 5. Salvando o Melhor Modelo (Parte 4.1)\njoblib.dump(best_model, MODEL_PATH)\nprint(f"Modelo '{best_model_name}' salvo em {MODEL_PATH}")\n\n# 6. Demonstração de Uso (Parte 4.2)\nloaded_model = joblib.load(MODEL_PATH)\n\n# Pegando a primeira amostra do conjunto de teste para demonstração\nsample_X = X_test.iloc[0].values.reshape(1, -1)\nsample_y_true = y_test[0]\nsample_y_pred = loaded_model.predict(sample_X)[0]\n\nprint(f"\nPrevisão de Demonstração (Modelo: {best_model_name}):")\nprint(f"  Valor Real: R$ {sample_y_true:,.2f}")\nprint(f"  Previsão do Modelo: R$ {sample_y_pred:,.2f}")\n""")
print("Script Python 03_Modelagem_e_Avaliacao.py salvo para execução.")